## Ray

#### Qu'est-ce que Ray ?

Pendant tout ce cours, nous avons utilisé le pack Ray :

In [1]:
import ray.rllib

![](img/ray-logo.png)

Qu'est-ce que Ray ? Dans les [docs] (https://docs.ray.io/en/latest/) :

&gt; Ray est un cadre de calcul distribué universel et polyvalent.

Ray est aussi :

- Un [projet open source actif](https://github.com/ray-project/ray) avec plus de 20k étoiles sur GitHub 🤩
- Soutenu par la startup licorne [Anyscale](https://www.anyscale.com/), qui a produit ce cours 🦄

Notes :

Mais, revenons à l'informatique distribuée.

#### Qu'est-ce que l'informatique distribuée ?

l'informatique distribuée est une informatique qui implique plusieurs machines (nœuds) réparties sur un réseau.

[](img/supercomputer.png)

Pour :

- Capacités massivement améliorées

Contre/défis :

- Synchronisation
- Échec
- ...

#### Ray facilite l'informatique distribuée

- L'objectif de Ray est de rendre l'informatique distribuée facile et accessible.
- Ray gère la plupart des défis pour les utilisateurs.
- RLlib, tune et les autres sous-paquets ont été construits au-dessus de Ray.
- Cela signifie que _RLlib et tune ont automatiquement des capacités distribuées._

Notes :

Surprise ! RLlib est facile à utiliser et implémente de manière pratique de nombreux algorithmes RL de pointe, mais il présente un autre avantage que nous n'avions pas mentionné jusqu'à présent : des capacités naturelles de calcul distribué. Cela le place loin devant les paquets concurrents en matière de facilité de distribution des calculs.

#### RLlib, distribué

- Dans ce cours, nous avons configuré les algorithmes de nombreuses fois.
- Mais il y a certains paramètres que nous n'avons jamais utilisés auparavant :

In [2]:
from ray.rllib.algorithms.ppo import PPOConfig

In [3]:
ppo_config = (
    PPOConfig()
    .framework("torch")
    .rollouts(num_rollout_workers=4, num_envs_per_worker=2)
    .resources(num_gpus=0)
)

Tu peux en savoir plus sur la spécification des ressources [ici] (https://docs.ray.io/en/master/rllib/rllib-training.html#specifying-resources) et sur la mise à l'échelle [ici] (https://docs.ray.io/en/master/rllib/rllib-training.html#scaling-guide).

Mais... qu'est-ce qu'un "ouvrier roulant" ?

#### Travailleurs du déploiement

- Les travailleurs du déploiement collectent les données de l'environnement (simulateur) en parallèle.
- Pour la plupart des environnements de simulateur, on peut répliquer l'environnement dans un cluster.
- Par conséquent, tu peux collecter les données beaucoup plus rapidement et éviter d'engorger la formation.
- Quel que soit le cluster auquel Ray est connecté en backend, `num_rollout_workers=4` fonctionne sans problème.

Remarques :

Dans l'apprentissage supervisé, quand tu attends, tu sais que tu attends probablement que le modèle s'entraîne. En RL, le goulot d'étranglement peut être la collecte de données ou les mises à jour du modèle. Le fait de pouvoir paralléliser les déploiements atténue le goulot d'étranglement de la collecte de données 

#### L'air de Ray, revisité

- N'oublie pas que le réglage des hyperparamètres, comme la recherche sur grille, est également facile à distribuer.
- Heureusement, `tune` fait aussi partie de Ray et, comme RLlib, s'en occupe pour toi ! 
- Comme tu peux le voir, Ray + tune + RLlib devient une combinaison assez puissante.

#### Conducteur

Dans toutes nos configurations, nous avons eu

```python
create_env_on_driver = True
```

Ce que cela signifie, c'est que nous plaçons l'env sur le même processus "pilote" qui exécute l'entraînement.

#### Résumé

- Ray est incroyablement puissant, et nous n'avons fait qu'effleurer la pointe de l'iceberg.
- Quelques autres ressources :
  - [ray.io](https://www.ray.io/) (en anglais)
  - [Learning Ray](https://www.oreilly.com/library/view/learning-ray/9781098117214/) (livre)

#### Appliquons ce que nous avons appris !

## Qu'est-ce que Ray ?
<!-- multiple choice -->

Qu'est-ce que Ray ?

- [ ] L'entreprise qui crée RLlib. | Tu penses peut-être à Anyscale, l'entreprise qui se cache derrière Ray !
- [ ] Un sous-paquet de RLlib qui traite de l'informatique distribuée.
- [ ] Un paquetage polyvalent qui inclut RLlib et qui traite du calcul distribué.
- [ ] Un algorithme d'apprentissage par renforcement 

## Distribuer RLlib
<!-- multiple choice -->

Quelle est la principale façon dont RLlib utilise les capacités de calcul distribué ?

- [x] Les travailleurs de déploiement distribués génèrent des données à partir des clones env qui sont introduites dans l'algorithme d'apprentissage 
- [ ] La formation du réseau neuronal de politique est répartie sur plusieurs nœuds.
- [ ] Chaque nœud possède un réseau neuronal de politique distinct qui est formé indépendamment sur son propre nœud.

## Expérimenter avec les travailleurs du déploiement
<!-- coding exercise -->

Le code ci-dessous crée deux instances de l'algorithme PPO, l'une qui devrait utiliser deux travailleurs de déploiement avec deux env par travailleur, et l'autre qui n'utilise qu'un seul travailleur de déploiement avec un env par travailleur. Il imprime ensuite le temps écoulé pour former chacune d'elles pendant 5 itérations. Termine et exécute le code, puis compare les temps 

Remarque : cette expérience fonctionnera _normalement_. Cependant, ce code s'exécute sur un serveur qui est potentiellement utilisé par plusieurs apprenants en même temps, les temps d'exécution peuvent donc être influencés par la charge du serveur. De plus, ce serveur n'est pas un véritable cluster, donc tout avantage proviendrait de la disponibilité de plusieurs cœurs de CPU pour la parallélisation dans une seule machine.

In [4]:
# EXERCISE

from ray.rllib.algorithms.ppo import PPOConfig
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

ppo_config_many = (
    PPOConfig()
    .framework("torch")
    .____(____)
    .training(model={"fcnet_hiddens" : [32,32]})
)

ppo_config_single = (
    PPOConfig()
    .framework("torch")
    .____(num_rollout_workers=1, num_envs_per_worker=1)
    .training(model={"fcnet_hiddens" : [32,32]})
)

ppo_many = ppo_config_many.build(env="FrozenLake-v1")
t = time.time()
for i in range(5):
    ppo_many.train()
print(f"Elapsed time with 2 workers, 2 envs each: {time.time()-t:.1f}s.")
ppo_many.stop()

ppo_single = ppo_config_single.build(env="FrozenLake-v1")
t = time.time()
for i in range(5):
    ppo_single.train()
print(f"Elapsed time with 1 worker, 1 env: {time.time()-t:.1f}s.")
ppo_single.stop()

AttributeError: 'PPOConfig' object has no attribute '____'

In [5]:
# SOLUTION

from ray.rllib.algorithms.ppo import PPOConfig
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

ppo_config_many = (
    PPOConfig()
    .framework("torch")
    .rollouts(num_rollout_workers=2, num_envs_per_worker=2)
    .training(model={"fcnet_hiddens" : [32,32]})
)

ppo_config_single = (
    PPOConfig()
    .framework("torch")
    .rollouts(num_rollout_workers=1, num_envs_per_worker=1)
    .training(model={"fcnet_hiddens" : [32,32]})
)

ppo_many = ppo_config_many.build(env="FrozenLake-v1")
t = time.time()
for i in range(5):
    ppo_many.train()
print(f"Elapsed time with 2 workers, 2 envs each: {time.time()-t:.1f}s.")
ppo_many.stop()

ppo_single = ppo_config_single.build(env="FrozenLake-v1")
t = time.time()
for i in range(5):
    ppo_single.train()
print(f"Elapsed time with 1 worker, 1 env: {time.time()-t:.1f}s.")
ppo_single.stop()

2022-08-27 17:02:42,927	INFO worker.py:1490 -- Started a local Ray instance.
(RolloutWorker pid=70484) 2022-08-27 17:02:47,263	WARNING env.py:154 -- Your env reset() method appears to take 'seed' or 'return_info' arguments. Note that these are not yet supported in RLlib. Seeding will take place using 'env.seed()' and the info dict will not be returned from reset.
(RolloutWorker pid=70484) 2022-08-27 17:02:47,536	WARNING deprecation.py:47 -- DeprecationWarning: `concat_samples` has been deprecated. Use `concat_samples() from rllib.policy.sample_batch` instead. This will raise an error in the future!


Elapsed time with 2 workers, 2 envs each: 11.3s.


(RolloutWorker pid=70561) 2022-08-27 17:03:00,941	WARNING env.py:154 -- Your env reset() method appears to take 'seed' or 'return_info' arguments. Note that these are not yet supported in RLlib. Seeding will take place using 'env.seed()' and the info dict will not be returned from reset.


Elapsed time with 1 worker, 1 env: 18.0s.
